In [2]:
!pip install -U wandb evaluate huggingface_hub datasets bert_score evaluate transformers numpy==1.26

   ---------------------------------------- 0.0/6.5 MB ? eta -:--:--
   - -------------------------------------- 0.3/6.5 MB ? eta -:--:--
   -------- ------------------------------- 1.3/6.5 MB 4.8 MB/s eta 0:00:02
   ---------------------- ----------------- 3.7/6.5 MB 8.1 MB/s eta 0:00:01
   ------------------------------------- -- 6.0/6.5 MB 9.2 MB/s eta 0:00:01
   ---------------------------------------- 6.5/6.5 MB 8.9 MB/s eta 0:00:00
  Attempting uninstall: wandb
    Found existing installation: wandb 0.17.6
    Uninstalling wandb-0.17.6:
      Successfully uninstalled wandb-0.17.6


In [3]:
import numpy as np
import pandas as pd
import torch
import os
from torch.utils.data import DataLoader, random_split, Dataset
from datasets import load_dataset

# USE RAY TUNE. https://docs.ray.io/en/latest/train/examples/intel_gaudi/bert.html
# USE E5SCORE AS A LOSS
# deepl translations

In [4]:
import wandb
from huggingface_hub import HfApi, HfFolder
from transformers import set_seed

try: # If it is on Kaggle
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()

    HF_TOKEN = user_secrets.get_secret("HF_TOKEN")
    WANDB_KEY = user_secrets.get_secret("WANDB_KEY")

except ModuleNotFoundError: # If it is local
    HF_TOKEN = os.environ["HF_TOKEN"]
    WANDB_KEY = os.environ["WANDB_KEY"]
    

HfFolder.save_token(HF_TOKEN)
wandb.login(key=WANDB_KEY)

# Reproducibility
seed = 1
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
set_seed(seed)
np.random.seed(seed)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: abdulmohsena. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\user\.netrc


## Modeling

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, GenerationConfig
from transformers import DataCollatorForSeq2Seq

In [6]:
# # Configure any model from HF HUB
# assert input("YOU WILL REMOVE THE HUB MODEL FOR THIS, TYPE 'OK' TO PROCEED: ").upper() == 'OK'
# model_name = "facebook/mbart-large-50-many-to-many-mmt"
# model_name = "facebook/m2m100_1.2B"
# #model_name= "Helsinki-NLP/opus-mt-en-ar"
# model_name= "facebook/nllb-200-distilled-600M"

# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
# generation_config = GenerationConfig(
#     max_length=200,
#     forced_bos_token_id=256011, # Arabic
#     num_beams = 4,
#     early_stopping=True,
#     do_sample=True,
#     top_k=50,
    
#     # Testing Config
# #     num_return_sequences=4, # Number of sentences to generate
# #     return_dict_in_generate=True, # Returns the complete generation data from within the model.
# #     output_scores=True, # Score of each token.
# )

# tokenizer.src_lang="eng_Latn"
# tokenizer.tgt_lang="arb_Arab"

# model.push_to_hub("Abdulmohsena/Faseeh")
# tokenizer.push_to_hub("Abdulmohsena/Faseeh")

In [7]:
# Instantiating The Model
model_name = "Abdulmohsena/Faseeh"

tokenizer = AutoTokenizer.from_pretrained(model_name, src_lang="eng_Latn", tgt_lang="arb_Arab")
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
generation_config = GenerationConfig.from_pretrained(model_name)

# https://huggingface.co/docs/transformers/en/main_classes/text_generation

In [8]:
# Sanity Check
dummy = "And the Saudi Arabian Foreign Minister assured the visitors that security is always a top priority."

encoded_ar = tokenizer(dummy, return_tensors="pt")
generated_tokens = model.generate(**encoded_ar, generation_config=generation_config)

tokenizer.decode(generated_tokens[0], skip_special_tokens=True)

'وآمن وزير خارجية السعودية الزوار أن الأمن دائم الأولوية القصوى.'

In [9]:
dataset = load_dataset("Abdulmohsena/Classic-Arabic-English-Language-Pairs")

dataset = dataset['train']

In [10]:
preprocess_function = lambda examples: tokenizer(
        examples['source'], text_target=examples['target'], max_length=128, truncation=True, padding=True)

tokenized_dataset = dataset.map(preprocess_function, batched=True)
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.25)

In [11]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_name)

# from transformers import DataCollatorForLanguageModeling # NOT GOOD FOR SEMANTIC TRANSLATION

# data_collator = DataCollatorForLanguageModeling(
#     tokenizer=tokenizer,
#     mlm=True,             # Whether to use Masked Language Modeling (MLM)
#     mlm_probability=0.15  # Probability of masking tokens for MLM
# )

In [12]:
import numpy as np
import evaluate
import transformers

metric = evaluate.load("bertscore")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    return preds, labels

def compute_metrics(eval_preds): 
    
    preds, labels = eval_preds
    
    # Replace unknown labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    
    # Decode tokens into text
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Postprocess text for cleaniness
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    # Get Average bertscore F-1
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, lang="ar")
    result = {"bertscore-f1": np.mean(result['f1'])}

    # Get avg gen length
    prediction_lengths = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lengths)

    result = {k: round(v, 4) for k, v in result.items()} # Round to 4 figures

    return result

## Training

In [13]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
train_batch_size = 2
torch.cuda.empty_cache()

# https://huggingface.co/docs/transformers/v4.42.0/performance
training_args = Seq2SeqTrainingArguments(
    output_dir=f"Faseeh_{metric.name}",
    save_total_limit=1,
    
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=train_batch_size,
    gradient_accumulation_steps=4 // train_batch_size,
    gradient_checkpointing=True,
    # torch_compile=True,
    
    logging_strategy="steps",
    logging_steps=500, 
    
    eval_strategy='epoch',
    
    weight_decay=0.01,
    warmup_steps=1_000,
    learning_rate=3e-5,
    lr_scheduler_type="cosine",
    
    
    num_train_epochs=1,
    
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
    report_to='wandb'
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

d:\anaconda\envs\Faseeh\Lib\site-packages\accelerate\accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [14]:
from datetime import datetime

wandb.init(project="Faseeh", name=f"Run @ {datetime.now()}")
trainer.train()
wandb.finish()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
  0%|          | 0/15544 [00:00<?, ?it/s]d:\anaconda\envs\Faseeh\Lib\site-packages\torch\_dynamo\eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
d:\anaconda\envs\Faseeh\Lib\site-packages\torch\utils\checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', a

KeyboardInterrupt: 